<a href="https://colab.research.google.com/github/Alessandra54/MVP_MachineLearning/blob/main/MVP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MVP – Engenharia de Software para Sistemas Inteligentes

## Contextualização do Projeto

Este projeto foi desenvolvido como parte do MVP da disciplina de Engenharia de Software para Sistemas Inteligentes, com o objetivo de aplicar técnicas de machine learning supervisionado para resolução de um problema de classificação, seguindo as boas práticas estudadas ao longo do curso.

## Descrição do Dataset

O dataset selecionado para este trabalho é o **World University Rankings**, disponível publicamente no repositório Kaggle. Este conjunto de dados apresenta um panorama abrangente das principais universidades ao redor do mundo, incluindo informações como:

- Nome e localização geográfica da universidade;
- Número total de estudantes;
- Número de estudantes internacionais;
- Classificações globais e regionais;
- Indicadores de desempenho acadêmico.

A escolha desse dataset justifica-se por sua riqueza de atributos e aplicabilidade em cenários reais de análise educacional, possibilitando a construção de um modelo de classificação que categoriza as universidades com base em características institucionais. O uso desse tipo de dado também permite discutir desafios comuns em problemas reais de ciência de dados, como o pré-processamento, a transformação de variáveis e a escolha de algoritmos adequados para tarefas de classificação.

-> Neste projeto iremos classificar universidades por "nível de desempenho" com base na overall_score

# Importando a URL

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import numpy as np


In [ ]:
url = 'https://raw.githubusercontent.com/Alessandra54/MVP_MachineLearning/refs/heads/main/World%20University%20Rankings.csv'

df = pd.read_csv(url)

df.head()

,name,scores_teaching,scores_research,scores_citations,scores_industry_income,scores_international_outlook,record_type,member_level,location,stats_number_students,stats_student_staff_ratio,stats_pc_intl_students,stats_female_male_ratio,subjects_offered,closed,unaccredited,overall_score
0,University of Oxford,96.6,100.0,99.0,98.7,97.5,master_account,0,United Kingdom,"21,750",10.9,42%,49:51:00,"Geography,Chemistry,Chemical Engineering,Biolo...",False,False,98.5
1,Stanford University,99.0,97.8,99.6,100.0,87.0,private,0,United States,"14,517",6.4,23%,47:53:00,"Computer Science,Communication & Media Studies...",False,False,98
2,Massachusetts Institute of Technology,98.6,96.2,99.7,100.0,93.8,private,0,United States,"11,085",8.0,33%,41:59:00,"Architecture,Economics & Econometrics,Archaeol...",False,False,97.9
3,Harvard University,97.7,99.9,99.4,84.2,90.8,private,0,United States,"20,050",9.0,25%,51:49:00,"Sociology,Architecture,Physics & Astronomy,Psy...",False,False,97.8
4,University of Cambridge,95.8,100.0,98.0,87.9,97.4,master_account,0,United Kingdom,"20,565",11.5,38%,48:52:00,Politics & International Studies (incl Develop...,False,False,97.5


In [ ]:
url = 'https://raw.githubusercontent.com/Alessandra54/MVP_MachineLearning/refs/heads/main/World%20University%20Rankings.csv'

df = pd.read_csv(url)

df.head()

df = df[df['overall_score'].notna()]  # remover nulos antes
df['overall_score'] = pd.to_numeric(df['overall_score'], errors='coerce')
df['desempenho'] = pd.qcut(df['overall_score'], q=3, labels=['Baixo', 'Médio', 'Alto'])
print(df[['overall_score', 'desempenho']].head())

   overall_score desempenho
0           98.5       Alto
1           98.0       Alto
2           97.9       Alto
3           97.8       Alto
4           97.5       Alto


# **Treino e teste (Holdout)**


In [ ]:
from sklearn.model_selection import train_test_split

X = df[['location', 'overall_score']]
y = df['desempenho']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# **Normalização e Padronização**


In [ ]:
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, StandardScaler
from sklearn.compose import ColumnTransformer

# Colunas
cat_cols = ['location']
num_cols = ['overall_score']

# Normalização MinMax
preprocess_minmax = ColumnTransformer([
    ('onehot', OneHotEncoder(handle_unknown='ignore'), cat_cols),
    ('minmax', MinMaxScaler(), num_cols)
])

# Padronização StandardScaler
preprocess_standard = ColumnTransformer([
    ('onehot', OneHotEncoder(handle_unknown='ignore'), cat_cols),
    ('standard', StandardScaler(), num_cols)
])

# **Pipelines**

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.neighbors import KNeighborsClassifier

idx_validos = y_train.notna()
X_train = X_train.loc[idx_validos]
y_train = y_train.loc[idx_validos]

cat_cols = ['location']
num_cols = ['overall_score']

# Pipeline numérico: imputar mediana + normalizar
num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', MinMaxScaler())
])

# Pipeline categórico: one-hot
cat_pipeline = Pipeline([
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessador = ColumnTransformer([
    ('num', num_pipeline, num_cols),
    ('cat', cat_pipeline, cat_cols)
])

pipeline_knn = Pipeline([
    ('preprocess', preprocessador),
    ('model', KNeighborsClassifier())
])


In [ ]:
# 4. Criar transformer para converter sparse -> dense (para Naive Bayes)
class DenseTransformer(TransformerMixin):
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        return X.toarray()

dense_transformer = DenseTransformer()

In [ ]:
# 5. Criar pipelines para cada modelo

# KNN
pipeline_knn = Pipeline([
    ('preprocess', preprocessador),
    ('model', KNeighborsClassifier())
])

# Árvore de Decisão
pipeline_tree = Pipeline([
    ('preprocess', preprocessador),
    ('model', DecisionTreeClassifier(random_state=42))
])

# Naive Bayes (com conversão para dense)
pipeline_nb = Pipeline([
    ('preprocess', preprocessador),
    ('to_dense', dense_transformer),
    ('model', GaussianNB())
])

# SVM
pipeline_svm = Pipeline([
    ('preprocess', preprocessador),
    ('model', SVC())
])

# 6. Avaliar todos os modelos com cross-validation
modelos = {
    'KNN': pipeline_knn,
    'Árvore de Decisão': pipeline_tree,
    'Naive Bayes': pipeline_nb,
    'SVM': pipeline_svm
}

for nome, pipe in modelos.items():
    scores = cross_val_score(pipe, X_train, y_train, cv=3)
    print(f'{nome} - Acurácia média (CV=3): {scores.mean():.4f}')


KNN - Acurácia média (CV=3): 0.7805
Árvore de Decisão - Acurácia média (CV=3): 0.9817
Naive Bayes - Acurácia média (CV=3): 0.4211
SVM - Acurácia média (CV=3): 0.5853


# **Avaliação dos modelos**

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Corrigir tipos
y_train = y_train.astype(str)
y_test = y_test.astype(str)

# Avaliação
resultados = {}

for nome, modelo in modelos.items():
    modelo.fit(X_train, y_train)
    y_pred = modelo.predict(X_test)

    # Calcular métricas
    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred, average='weighted')
    rec = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')

    resultados[nome] = {
        'Accuracy': acc,
        'Precision': prec,
        'Recall': rec,
        'F1-score': f1
    }

# Mostrar resultados
df_resultados = pd.DataFrame(resultados).T
print(df_resultados)



                   Accuracy  Precision    Recall  F1-score
KNN                0.065617   0.010984  0.065617  0.018312
Árvore de Decisão  0.097113   0.059388  0.097113  0.060901
Naive Bayes        0.023622   0.000844  0.023622  0.001629
SVM                0.044619   0.012703  0.044619  0.017608


In [ ]:
melhor_modelo_nome = df_resultados['F1-score'].idxmax()
print(f'Melhor modelo: {melhor_modelo_nome}')

Melhor modelo: Árvore de Decisão


# **Exportar modelo**

In [ ]:
import joblib
import sklearn
print(sklearn.__version__)

# Recupera o pipeline já treinado
melhor_modelo = modelos[melhor_modelo_nome]
melhor_modelo.fit(X_train, y_train)

# Salvar
joblib.dump(melhor_modelo, f'{melhor_modelo_nome}_pipeline1.pkl')
print(f'Modelo salvo como {melhor_modelo_nome}_pipeline1.pkl')


1.6.1
Modelo salvo como Árvore de Decisão_pipeline1.pkl
